# X3D (eXpanded 3D) Implementation - Design Alternative 1

`Warning:` Do not run all the code, read the notes first before running them as they may reduce your device's memory when not careful, leading to a kernel crash or Out-Of-Memory (OOM) Error, sige ka magla-lag yang pc mo

`Note`: Isipin mo alternate universe na training toh

**Local System Specifications used to train the model:** <br>
| Component  | Device   |
| :------------- | :--------------- | 
| CPU   | AMD Ryzen 5 5600x     | 
| GPU   | Nvidia GeForce RTX 3070  8GB   | 
| RAM   | 32 GB     | 


`Note`: This notebook's codes will come from [PytorchVideo](https://pytorchvideo.org/docs/tutorial_overview) and from this Talha Anwar's [YouTube Playlist](https://www.youtube.com/playlist?list=PLtGXgNsNHqPRAscIi6dMUuPCfDQ1Vho2U)

#### Import Libraries

In [1]:
import sys
sys.path.append('pytorchvideo')
import torchvision.transforms.functional as F_t

sys.modules["torchvision.transforms.functional_tensor"] = F_t
import pytorchvideo
from pytorchvideo.data import *
import pytorch_lightning
import torchmetrics
import torch

import pandas as pd
import pathlib
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

In [2]:
pathlib.PosixPath = pathlib.WindowsPath # For windows only

#### Prepare the dataset

In [3]:
dataset_dir = pathlib.Path(r"./PD_Dataset")

In [4]:
def renameFile():
    for file_path in dataset_dir.rglob("*.avi"):
        # Get the parent directory name and clean it up
        dir_name_clean = file_path.parent.name.replace(" ", "_")
        # Clean the file name by replacing spaces with underscores
        file_clean = file_path.name.replace(" ", "_")
        
        # Create the new file name
        new_name = f"{dir_name_clean}_{file_clean}"
        new_path = file_path.parent / new_name
        
        print(f"Renaming: {file_path} -> {new_path}")
        file_path.rename(new_path)

In [5]:
#renameFile() #run once...

In [6]:
categories = ["BarbellCurl", "Deadlift", "LateralRaises", "OverheadPress", "Squat"]

video_labels = [
    [video for video in (dataset_dir / category).iterdir() if video.is_file()]
    for category in categories
]

# Flatten and create labels
label = [i for i, videos in enumerate(video_labels) for _ in videos]
combined = [video for sublist in video_labels for video in sublist]

print(label)
print(combined)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 

In [7]:
df = pd.DataFrame(zip(combined, label), columns=["file", "label"])
df.head()

,file,label
0,PD_Dataset\BarbellCurl\BarbellCurl_barbell_cur...,0
1,PD_Dataset\BarbellCurl\BarbellCurl_barbell_cur...,0
2,PD_Dataset\BarbellCurl\BarbellCurl_barbell_cur...,0
3,PD_Dataset\BarbellCurl\BarbellCurl_barbell_cur...,0
4,PD_Dataset\BarbellCurl\BarbellCurl_barbell_cur...,0


In [8]:
df["label"].value_counts()

label
3    157
4    132
2    125
1    124
0    123
Name: count, dtype: int64

data is not balanced so... Remove tayo randomly :)

In [9]:
## Function to balance
def balance_dataset(df):
    # Find the minimum count among all classes
    min_count = df["label"].value_counts().min()
    
    # Downsample each class to match min_count
    balanced_df = (
        df.groupby("label").apply(lambda x: x.sample(n=min_count, random_state=43)).reset_index(drop=True)
    )
    
    return balanced_df

In [10]:
df = balance_dataset(df)
df["label"].value_counts() # Check for the number of data

C:\Users\kuron\AppData\Local\Temp\ipykernel_10560\830260054.py:8: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df.groupby("label").apply(lambda x: x.sample(n=min_count, random_state=43)).reset_index(drop=True)


label
0    123
1    123
2    123
3    123
4    123
Name: count, dtype: int64

#### Split the dataset

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

train_df, val_df = train_test_split(df, test_size=0.3, shuffle=True)
val_df, test_df = train_test_split(val_df, test_size=0.1, shuffle=False)

In [12]:
len(train_df), len(val_df), len(test_df)

(430, 166, 19)

In [13]:
train_df.to_csv("train_df.csv", header=False , sep=" ", index=False) 
val_df.to_csv("val_df.csv", header=False, sep=" ", index=False)
test_df.to_csv("test_df.csv", header=False, sep=" ", index=False)

#### Prepare and augment the dataset for better generalization of the mudel 

In [14]:
from pytorchvideo.transforms import (
    ApplyTransformToKey,
    Normalize, RandomShortSideScale,
    UniformTemporalSubsample, Permute
)

from torchvision.transforms import (
    Compose, Lambda, RandomCrop,
    RandomHorizontalFlip, Resize,
    ToTensor, ToPILImage
    
)

from torchvision.transforms._transforms_video import (
    CenterCropVideo, NormalizeVideo
)

c:\Users\kuron\vscodeProjects\myenv\Lib\site-packages\torchvision\transforms\_functional_video.py:6: UserWarning: The 'torchvision.transforms._functional_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms.functional' module instead.
  warnings.warn(
c:\Users\kuron\vscodeProjects\myenv\Lib\site-packages\torchvision\transforms\_transforms_video.py:22: UserWarning: The 'torchvision.transforms._transforms_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms' module instead.
  warnings.warn(


In [17]:
video_transform = Compose([
    ApplyTransformToKey(key='video',
    transform=Compose([
        UniformTemporalSubsample(20),
        Lambda(lambda x: x/255.0),
        Normalize((0.45, 0.45, 0.45), (0.225, 0.225, 0.225)),
        RandomShortSideScale(min_size=112, max_size=128),
        CenterCropVideo(112),
        RandomHorizontalFlip(p=0.5),
    ])
    )
])

In [18]:
train_dataset = labeled_video_dataset("train_df.csv",
                                      clip_sampler=make_clip_sampler('random', 2),
                                      transform=video_transform,
                                      decode_audio=False
                                      )
loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=8,
    num_workers=0,
    pin_memory=True
)

batch = next(iter(loader))
print(batch.keys())
print(batch['video'].shape, batch['label'].shape)

dict_keys(['video', 'video_name', 'video_index', 'clip_index', 'aug_index', 'label'])
torch.Size([8, 3, 20, 112, 112]) torch.Size([8])


In [19]:
video_model = torch.hub.load('facebookresearch/pytorchvideo', 'x3d_s', pretrained=True)
video_model

Using cache found in C:\Users\kuron/.cache\torch\hub\facebookresearch_pytorchvideo_main


Net(
  (blocks): ModuleList(
    (0): ResNetBasicStem(
      (conv): Conv2plus1d(
        (conv_t): Conv3d(3, 24, kernel_size=(1, 3, 3), stride=(1, 2, 2), padding=(0, 1, 1), bias=False)
        (conv_xy): Conv3d(24, 24, kernel_size=(5, 1, 1), stride=(1, 1, 1), padding=(2, 0, 0), groups=24, bias=False)
      )
      (norm): BatchNorm3d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activation): ReLU()
    )
    (1): ResStage(
      (res_blocks): ModuleList(
        (0): ResBlock(
          (branch1_conv): Conv3d(24, 24, kernel_size=(1, 1, 1), stride=(1, 2, 2), bias=False)
          (branch2): BottleneckBlock(
            (conv_a): Conv3d(24, 54, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
            (norm_a): BatchNorm3d(54, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (act_a): ReLU()
            (conv_b): Conv3d(54, 54, kernel_size=(3, 3, 3), stride=(1, 2, 2), padding=(1, 1, 1), groups=54, bias=False)
            (nor

In [20]:
class VideoClassificationModel(pytorch_lightning.LightningModule):
    def __init__(self):
        super().__init__()
        self.num_classes = len(categories)
        self.video_model = torch.hub.load('facebookresearch/pytorchvideo', 'x3d_s', pretrained=True)
        self.relu = torch.nn.ReLU()
        self.linear = torch.nn.Linear(400, self.num_classes) # categories has a value of 5 because we have 5 classes

        self.lr = 1e-4
        self.batch_size = 8
        self.num_worker = 0
        self.metric = torchmetrics.Accuracy(task='multiclass', num_classes=self.num_classes)
        self.criterion = torch.nn.CrossEntropyLoss()

    def forward(self, x):
        x = self.video_model(x)
        x = self.relu(x)
        x = self.linear(x)
        return x
    
    def configure_optimizers(self):
        opt = torch.optim.AdamW(self.parameters(), lr=self.lr)
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(opt, T_max=10, eta_min=1e-6, last_epoch=-1)
        return {'optimizer': opt, 'lr_scheduler': scheduler}

    
    def train_dataloader(self):
        train_dataset = labeled_video_dataset("train_df.csv",
                                      clip_sampler=make_clip_sampler('random', 2),
                                      transform=video_transform,
                                      decode_audio=False)
        loader = torch.utils.data.DataLoader(
            train_dataset,
            batch_size=self.batch_size,
            num_workers=self.num_worker,
            pin_memory=True)
        
        _ = iter(loader) # Force iterarion
        return loader

    def training_step(self, batch, batch_idx):
        video, label = batch['video'], batch['label']
        out = self(video)
        loss = self.criterion(out, label)
        metric = self.metric(out, label.to(torch.int64))
        return {'loss': loss, 'metric': metric.detach()}
    
    def on_train_epoch_end(self, outputs):
        loss = torch.stack([x['loss'] for x in outputs]).mean().cpu().numpy().round(2)
        metric = torch.stack([x['metric'] for x in outputs]).mean().cpu().numpy().round(2)
        self.log('train_loss', loss)
        self.log('train_metric', metric)

    def val_dataloader(self):
        val_dataset = labeled_video_dataset("val_df.csv",
                                      clip_sampler=make_clip_sampler('random', 2),
                                      transform=video_transform,
                                      decode_audio=False)
        loader = torch.utils.data.DataLoader(
            val_dataset,
            batch_size=self.batch_size,
            num_workers=self.num_worker,
            pin_memory=True)
        
        _ = iter(loader)
        return loader
    
    def validation_step(self, batch, batch_idx):
        video, label = batch['video'], batch['label']
        out = self(video)
        loss = self.criterion(out, label)
        metric = self.metric(out, label.to(torch.int64))
        return {'loss': loss, 'metric': metric.detach()}
    
    def on_validation_epoch_end(self, outputs):
        loss = torch.stack([x['loss'] for x in outputs]).mean().cpu().numpy().round(2)
        metric = torch.stack([x['metric'] for x in outputs]).mean().cpu().numpy().round(2)
        self.log('train_loss', loss)
        self.log('train_metric', metric)

    def test_dataloader(self):
        test_dataset = labeled_video_dataset("test_df.csv",
                                      clip_sampler=make_clip_sampler('random', 2),
                                      transform=video_transform,
                                      decode_audio=False)
        loader = torch.utils.data.DataLoader(
            test_dataset,
            batch_size=self.batch_size,
            num_workers=self.num_worker,
            pin_memory=True)
        
        _ = iter(loader)
        return loader
    
    def test_step(self, *args, **kwargs):
        video, label = batch['video'], batch['label']
        out = self(video)
        return {'label': label.detach(), 'pred': out.detach()}
    
    def on_test_epoch_end(self, outputs):
        label = torch.cat([x['label'] for x in outputs]).cpu().numpy()
        pred = torch.cat([x['pred'] for x in outputs]).cpu().numpy()

        print(classification_report(label, pred))
        

In [21]:
import pytorch_lightning.callbacks

checkpoints_callback = pytorch_lightning.callbacks.ModelCheckpoint(
    monitor='val_loss',
    dirpath='checkpoints',
    filename='file',
    save_last=True)

lr_monitor = pytorch_lightning.callbacks.LearningRateMonitor(logging_interval='epoch')

In [22]:
model = VideoClassificationModel()
pytorch_lightning.seed_everything(0)

trainer = pytorch_lightning.Trainer(
    max_epochs=100,
    accelerator='gpu',
    devices=-1,
    precision=16,
    accumulate_grad_batches=2,
    enable_progress_bar=False,
    num_sanity_val_steps=0,
    callbacks=[lr_monitor, checkpoints_callback],
    limit_train_batches=36,
    limit_val_batches=12
)

Using cache found in C:\Users\kuron/.cache\torch\hub\facebookresearch_pytorchvideo_main
Seed set to 0
c:\Users\kuron\vscodeProjects\myenv\Lib\site-packages\lightning_fabric\connector.py:572: `precision=16` is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
c:\Users\kuron\vscodeProjects\myenv\Lib\site-packages\pytorch_lightning\trainer\connectors\logger_connector\logger_connector.py:76: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard su

In [23]:
trainer.fit(model)

You are using a CUDA device ('NVIDIA GeForce RTX 3070') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
c:\Users\kuron\vscodeProjects\myenv\Lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:654: Checkpoint directory C:\Users\kuron\vscodeProjects\Python\checkpoints exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name        | Type               | Params | Mode 
-----------------------------------------------------------
0 | video_model | Net                | 3.8 M  | train
1 | relu        | ReLU               | 0      | train
2 | linear      | Linear             | 2.0 K  | train
3 | metric      | MulticlassAccuracy | 0      | train
4 | criterion   | CrossEntropyLoss   | 0      | train
-------

RuntimeError: input image (T: 20 H: 4 W: 4) smaller than kernel size (kT: 13 kH: 5 kW: 5)